In [17]:
!pip install pymongo

  Running setup.py bdist_wheel for pymongo: started
  Running setup.py bdist_wheel for pymongo: finished with status 'done'
  Stored in directory: C:\Users\BIG DATA\AppData\Local\pip\Cache\wheels\41\12\7b\8a5763b32bbea5d7bca33dfbdb4740541ef17f5ca3f22b3be9
Successfully built pymongo


In [55]:
from pymongo import MongoClient
import pymongo
import pymysql
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pickle
import json
import datetime


In [63]:
def get_sql_url(conn,source):
    list_url = []
    sql = "SELECT url FROM gp3.cklist WHERE source ='%s' and tag ='N' order by url desc limit 235"
    cur = conn.cursor()
    cur.execute(sql % source)
    for i in cur:
        list_url.append(i[0])
    return list_url


def insert_mongo(client,jsObj):
    collection.insert_one(jsObj)

    
def update_sql(conn,url):
    sql  = "update gp3.cklist set tag = 'R' where url = '%s' "
    cur = conn.cursor()
    cur.execute(sql%url)
    cur.connection.commit()


conn = pymysql.connect(host='10.120.28.52', port=3306, user='gp3', passwd='Group_03', db='gp3')
list_url = get_sql_url(conn,'voa')
client = MongoClient('10.120.28.55', 27017)
db = client['news']
collection = db['news']


for url in list_url:
    update_sql(conn, url)
    
    res = requests.get(url)
    
    soup = BeautifulSoup(res.text,'lxml')
    
    articles_dict = {}
    
    for time in soup.select('span.date > time'):
        
        articles_dict['date'] = str(time).split("T")[0].split("=\"")[1]
    
    for title in soup.select('h1.pg-title'):
    
        articles_dict['title'] = title.text.strip("\n")

    for push in soup.select("span"):
        push.extract()
    
    for push in soup.select("div.wsw > div"):
        push.extract()
    
    contents_dict = {}
    
    try:
        contents = soup.find('div',{'class':'wsw'}).get_text().replace('\n', '').replace('\'', '').replace('\"', '').strip().split('. ')
    except:pass
    
    
    for page in range(len(contents)):
    
        contents_dict[str(page+1)] = contents[page]
        
    articles_dict['content'] = contents_dict
    
    articles_dict['source'] = 'VOA'
    
    articles_dict['url'] = url
    
    insert_mongo(client,articles_dict)
    
    #print(articles_dict)

In [57]:
type(articles_dict)

dict